<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0101EN-SkillsNetwork/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

# Watson Speech to Text Translator

Estimated time needed: **25** minutes

## Objectives

After completing this lab you will be able to:

*   Operate a Speech to Text Translator through an API


### Introduction

<p>In this notebook, you will learn to convert an audio file of an English speaker to text using a Speech to Text API. Then, you will translate the English version to a Spanish version using a Language Translator API. <b>Note:</b> You must obtain the API keys and endpoints to complete the lab.</p>


<div class="alert alert-block alert-info" style="margin-top: 20px">
<h2>Table of Contents</h2>
<ul>
    <li><a href="#ref0">Speech To Text</a></li>
    <li><a href="#ref1">Language Translator</a></li>
    <li><a href="#ref2">Exercise</a></li>
</ul>
</div>


In [1]:
#you will need the following library 
!pip install ibm_watson wget

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for ibm-watson: filename=ibm_watson-5.2.2-py3-none-any.whl size=403389 sha256=c358c3952b497625fa44aaeee9601d0b295b38a365af9c539b5e22cba22cd1af
  Stored in directory: c:\users\sandipan.dey\appdata\local\pip\cache\wheels\64\af\0b\1f0f7f72a47e98ebbd1119568cca47e480969ef9dd688c70d6
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-3.10.1-py3-none-any.whl size=61765 sha256=05d7e11ddb6c2e4bda30f1978f2ff2ee0dc9ac088f038b97a6dff8a884077b7e
  Stored in directory: c:\users\sandipan.dey\appdata\local\pip\cache\wheels\db\0c\7e\f377d38b5729ec554daa3f81a010d2b466ff045ee8216cecb3
Successfully built ibm-watson ibm-cloud-sdk-core
  Attempting uninstall: PyJWT
   

ERROR: sanic-jwt 1.6.0 has requirement pyjwt==2.0.0, but you'll have pyjwt 2.1.0 which is incompatible.
ERROR: rasa 2.2.5 has requirement regex<2020.10,>=2020.6, but you'll have regex 2018.1.10 which is incompatible.
ERROR: rasa 2.2.5 has requirement ruamel.yaml<0.17.0,>=0.16.5, but you'll have ruamel-yaml 0.15.87 which is incompatible.
ERROR: rasa 2.2.5 has requirement SQLAlchemy<1.4.0,>=1.3.3, but you'll have sqlalchemy 0.7.10 which is incompatible.
ERROR: rasa 2.2.5 has requirement tensorflow<2.4,>=2.3, but you'll have tensorflow 1.14.0 which is incompatible.
ERROR: rasa 2.2.5 has requirement tensorflow-estimator<2.4,>=2.3, but you'll have tensorflow-estimator 1.14.0 which is incompatible.


<h2 id="ref0">Speech to Text</h2>


<p>First we import <code>SpeechToTextV1</code> from <code>ibm_watson</code>. For more information on the API, please click on this <a href="https://cloud.ibm.com/apidocs/speech-to-text?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01&code=python">link</a>.</p>


In [2]:
from ibm_watson import SpeechToTextV1 
import json
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

<p>The service endpoint is based on the location of the service instance, we store the information in the variable URL. To find out which URL to use, view the service credentials and paste the url here.</p>


In [20]:
{
  "apikey": "UBC7fX_l9EUDhq5ukuZTOQ8QiBo4TrBzoelMfmlTzGW-",
  "iam_apikey_description": "Auto-generated for key 304a3a1b-64d1-44fe-ba13-0ffa9080b133",
  "iam_apikey_name": "Auto-generated service credentials",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Manager",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/096c0b9603054e10b1a4994316e2d8a1::serviceid:ServiceId-5b8a8642-c6ae-4271-9727-0e7fc46cade5",
  "url": "https://api.eu-gb.speech-to-text.watson.cloud.ibm.com/instances/31e251c6-a500-4fbf-9cbb-c5d0fd8915f9"
}

{'apikey': 'idrwla-RfY7fJzIH8YpJu2Y3XxnjSbYMQTTC3AeC7YNZ',
 'iam_apikey_description': 'Auto-generated for key 7b38b9bd-efe6-4c02-b970-ab3a834c2b54',
 'iam_apikey_name': 'Auto-generated service credentials',
 'iam_role_crn': 'crn:v1:bluemix:public:iam::::serviceRole:Manager',
 'iam_serviceid_crn': 'crn:v1:bluemix:public:iam-identity::a/096c0b9603054e10b1a4994316e2d8a1::serviceid:ServiceId-8de5c6b5-d754-4b3c-89a2-870c3ebc1316',
 'url': 'https://api.eu-gb.language-translator.watson.cloud.ibm.com/instances/c30a554c-e273-43ba-b15c-d3eb4bcf6993'}

In [39]:
import json
from os.path import join, dirname
from ibm_watson import SpeechToTextV1
from ibm_watson.websocket import RecognizeCallback, AudioSource
import threading
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator('UBC7fX_l9EUDhq5ukuZTOQ8QiBo4TrBzoelMfmlTzGW-')
service = SpeechToTextV1(authenticator=authenticator)
service.set_service_url('https://api.eu-gb.speech-to-text.watson.cloud.ibm.com/instances/31e251c6-a500-4fbf-9cbb-c5d0fd8915f9')

models = service.list_models().get_result()
print(json.dumps(models, indent=2))

model = service.get_model('en-US_BroadbandModel').get_result()
print(json.dumps(model, indent=2))

with open('speech.wav',
          'rb') as audio_file:
    print(json.dumps(
        service.recognize(
            audio=audio_file,
            content_type='audio/wav',
            timestamps=True,
            word_confidence=True).get_result(),
        indent=2))

{
  "models": [
    {
      "name": "es-MX_BroadbandModel",
      "rate": 16000,
      "language": "es-MX",
      "description": "Mexican Spanish broadband model.",
      "supported_features": {
        "custom_language_model": true,
        "speaker_labels": true
      },
      "url": "https://api.eu-gb.speech-to-text.watson.cloud.ibm.com/instances/31e251c6-a500-4fbf-9cbb-c5d0fd8915f9/v1/models/es-MX_BroadbandModel"
    },
    {
      "name": "en-US_BroadbandModel",
      "rate": 16000,
      "language": "en-US",
      "description": "US English broadband model.",
      "supported_features": {
        "custom_language_model": true,
        "speaker_labels": true
      },
      "url": "https://api.eu-gb.speech-to-text.watson.cloud.ibm.com/instances/31e251c6-a500-4fbf-9cbb-c5d0fd8915f9/v1/models/en-US_BroadbandModel"
    },
    {
      "name": "es-CL_BroadbandModel",
      "rate": 16000,
      "language": "es-CL",
      "description": "Chilean Spanish broadband model.",
      "supported

{
  "name": "en-US_BroadbandModel",
  "rate": 16000,
  "language": "en-US",
  "description": "US English broadband model.",
  "supported_features": {
    "custom_language_model": true,
    "speaker_labels": true
  },
  "url": "https://api.eu-gb.speech-to-text.watson.cloud.ibm.com/instances/31e251c6-a500-4fbf-9cbb-c5d0fd8915f9/v1/models/en-US_BroadbandModel"
}
{
  "result_index": 0,
  "results": [
    {
      "final": true,
      "alternatives": [
        {
          "transcript": "thunderstorms could produce large hail isolated tornadoes and heavy rain ",
          "confidence": 1.0,
          "timestamps": [
            [
              "thunderstorms",
              1.45,
              2.32
            ],
            [
              "could",
              2.32,
              2.55
            ],
            [
              "produce",
              2.55,
              3.01
            ],
            [
              "large",
              3.01,
              3.35
            ],
         

In [40]:
url_s2t = "https://api.eu-gb.speech-to-text.watson.cloud.ibm.com/instances/31e251c6-a500-4fbf-9cbb-c5d0fd8915f9"

<p>You require an API key, and you can obtain the key on the <a href="https://cloud.ibm.com/resources?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01">Dashboard </a>.</p>


In [41]:
iam_apikey_s2t = "UBC7fX_l9EUDhq5ukuZTOQ8QiBo4TrBzoelMfmlTzGW-"

<p>You create a <a href="http://watson-developer-cloud.github.io/python-sdk/v0.25.0/apis/watson_developer_cloud.speech_to_text_v1.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01">Speech To Text Adapter object</a> the parameters are the endpoint and API key.</p>


In [42]:
authenticator = IAMAuthenticator(iam_apikey_s2t)
s2t = SpeechToTextV1(authenticator=authenticator)
s2t.set_service_url(url_s2t)
s2t

<p>Lets download the audio file that we will use to convert into text.</p>


In [ ]:
#!wget -O PolynomialRegressionandPipelines.mp3  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0101EN-SkillsNetwork/labs/Module%205/data/PolynomialRegressionandPipelines.mp3


<p>We have the path of the .wav file we would like to convert to text</p>


In [43]:
filename='PolynomialRegressionandPipelines.mp3'

<p>We create the file object <code>wav</code> with the wav file using <code>open</code>. We set the <code>mode</code> to  "rb" ,  this is similar to read mode, but it ensures the file is in binary mode. We use the method <code>recognize</code> to return the recognized text. The parameter <code>audio</code> is the file object <code>wav</code>, the parameter <code>content_type</code> is the format of the audio file.</p>


In [44]:
with open(filename, mode="rb")  as wav:
    response = s2t.recognize(audio=wav, content_type='audio/mp3')

<p>The attribute result contains a dictionary that includes the translation:</p>


In [45]:
response.result

{'result_index': 0,
 'results': [{'final': True,
   'alternatives': [{'transcript': 'in this video we will cover polynomial regression and pipelines ',
     'confidence': 0.94}]},
  {'final': True,
   'alternatives': [{'transcript': "what do we do when a linear model is not the best fit for our data let's look into another type of regression model the polynomial regression we transform our data into a polynomial then use linear regression to fit the parameters that we will discuss pipelines pipelines are way to simplify your code ",
     'confidence': 0.9}]},
  {'final': True,
   'alternatives': [{'transcript': "polynomial regression is a special case of the general linear regression this method is beneficial for describing curvilinear relationships what is a curvilinear relationship it's what you get by squaring or setting higher order terms of the predictor variables in the model transforming the data the model can be quadratic which means the predictor variable in the model is squar

In [46]:
from pandas import json_normalize

json_normalize(response.result['results'],"alternatives")

,transcript,confidence
0,in this video we will cover polynomial regress...,0.94
1,what do we do when a linear model is not the b...,0.90
2,polynomial regression is a special case of the...,0.95
3,the model can be cubic which means the predict...,0.95
4,there also exists higher order polynomial regr...,0.91
5,let's look at an example from our data we gene...,0.89
6,in python we do this by using the poly fit fun...,0.92
7,negative one point five five seven X. one cute...,0.90
8,consider the feature shown here applying the m...,0.90
9,pipeline sequentially perform a series of tran...,0.89


In [47]:
response

<p>We can obtain the recognized text and assign it to the variable <code>recognized_text</code>:</p>


In [48]:
recognized_text=response.result['results'][0]["alternatives"][0]["transcript"]
type(recognized_text)

str

In [58]:
recognized_text

'in this video we will cover polynomial regression and pipelines '

<h2 id="ref1">Language Translator</h2>


<p>First we import <code>LanguageTranslatorV3</code> from ibm_watson. For more information on the API click <a href="https://cloud.ibm.com/apidocs/speech-to-text?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01&code=python"> here</a></p>


In [50]:
from ibm_watson import LanguageTranslatorV3

<p>The service endpoint is based on the location of the service instance, we store the information in the variable URL. To find out which URL to use, view the service credentials.</p>


In [51]:
url_lt='https://api.eu-gb.language-translator.watson.cloud.ibm.com/instances/c30a554c-e273-43ba-b15c-d3eb4bcf6993'

<p>You require an API key, and you can obtain the key on the <a href="https://cloud.ibm.com/resources?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01">Dashboard</a>.</p>


In [52]:
apikey_lt='idrwla-RfY7fJzIH8YpJu2Y3XxnjSbYMQTTC3AeC7YNZ'

<p>API requests require a version parameter that takes a date in the format version=YYYY-MM-DD. This lab describes the current version of Language Translator, 2018-05-01</p>


In [53]:
version_lt='2018-05-01'

<p>we create a  Language Translator object <code>language_translator</code>:</p>


In [54]:
authenticator = IAMAuthenticator(apikey_lt)
language_translator = LanguageTranslatorV3(version=version_lt,authenticator=authenticator)
language_translator.set_service_url(url_lt)
language_translator

<p>We can get a Lists the languages that the service can identify.
The method Returns the language code.  For example English (en) to  Spanis (es) and name of each language.</p>


In [62]:
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.json_normalize(language_translator.list_identifiable_languages().get_result(), "languages")

,language,name
0,af,Afrikaans
1,ar,Arabic
2,az,Azerbaijani
3,ba,Bashkir
4,be,Belarusian
5,bg,Bulgarian
6,bn,Bengali
7,ca,Catalan
8,cs,Czech
9,cv,Chuvash


<p>We can use the method <code>translate</code>. This will translate the text. The parameter text is the text, Model_id is the type of model we would like to use use we use list the language. In this case, we set it to 'en-es' or English to Spanish. We get a Detailed Response object translation_response</p>


In [56]:
translation_response = language_translator.translate(\
    text=recognized_text, model_id='en-es')
translation_response

<p>The result is a dictionary.</p>


In [57]:
translation=translation_response.get_result()
translation

{'translations': [{'translation': 'en este vídeo cubriremos la regresión polinómica y las tuberías '}],
 'word_count': 10,
 'character_count': 64}

<p>We can obtain the actual translation as a string as follows:</p>


In [59]:
spanish_translation =translation['translations'][0]['translation']
spanish_translation 

'en este vídeo cubriremos la regresión polinómica y las tuberías '

<p>We can translate back to English</p>


In [60]:
translation_new = language_translator.translate(text=spanish_translation ,model_id='es-en').get_result()

<p>We can obtain the actual translation as a string as follows:</p>


In [61]:
translation_eng=translation_new['translations'][0]['translation']
translation_eng

'in this video we will cover the polynomial regression and the pipes '

<br>


<h2>Quiz</h2>


Translate to French.


In [ ]:
# Write your code below and press Shift+Enter to execute


<details><summary>Click here for the solution</summary>

```python
French_translation=language_translator.translate(
    text=translation_eng , model_id='en-fr').get_result()

French_translation['translations'][0]['translation']

```

</details>


<h3>Language Translator</h3>


<b>References</b>


https://cloud.ibm.com/apidocs/speech-to-text?code=python


https://cloud.ibm.com/apidocs/language-translator?code=python


<hr>


## Authors:

[Joseph Santarcangelo](https://www.linkedin.com/in/joseph-s-50398b136/?utm_medium=Exinfluencer\&utm_source=Exinfluencer\&utm_content=000026UJ\&utm_term=10006555\&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01)

Joseph Santarcangelo has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD.

## Other Contributor(s)

<a href="https://www.linkedin.com/in/fanjiang0619/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01">Fan Jiang</a>

## Change Log

|  Date (YYYY-MM-DD) |  Version | Changed By  |  Change Description |
|---|---|---|---|
| 2021-04-07 | 2.2 | Malika | Updated the libraries |
| 2021-01-05 | 2.1 | Malika | Added a library |
| 2020-08-26  | 2.0  | Lavanya | Moved lab to course repo in GitLab |
|   |   |   |   |
|   |   |   |   |

<hr/>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
